In [1]:
import sys,importlib
sys.path.append('../')
import simulation
from numpy import *
import plotly.offline as py
import plotly.graph_objs as go
import pandas

In [2]:
master = pandas.read_csv('./fort_longer.123',sep='\t')

In [4]:
master['dX'] = master['x'].diff()
master['dY'] = master['y'].diff()
master['dZ'] = master['z'].diff()

In [5]:
trace = go.Scatter3d(x=master['dX'],y=master['dY'],z=master['dZ'],mode='lines')
py.iplot([trace])

#### synchrony

$\phi = x_M - x_S - D \\$
$D = a_0 + a_1 sin(\omega t)$

In [140]:
a0,a1,w = 1.5,2.5,2*pi
epsilon = 2.5
delTime = .1

def zetaInvar(stateA,stateB,t):
    return asarray([-w*a1*cos(w*t)]*len(stateA))
def chi(stateA,stateB,t):
    return stateA-stateB-a0-a1*sin(w*t)
def zetaNull(stateA,stateB,t):
    return asarray([0,0,0],dtype=float)

## slave evolution

In [141]:
def slaveEvolution(master,zeta):
    slave = zeros((len(master),3))
    stateB = master[['x','y','z']].iloc[0].to_numpy()
    slave[0] = stateB
    for index,master_state in master.iloc[1:].iterrows():
        time = master_state['t']
        stateB = slave[index-1]
        stateA = master_state[['x','y','z']].to_numpy()

        dA = master_state[['dX','dY','dZ']].to_numpy()
        coupling = zeta(stateA,stateB,time) 
        coupling += chi(stateA,stateB,time)*epsilon
        stateB += delTime*(dA+coupling)
        slave[index] = stateB
    slave = pandas.DataFrame(slave,columns=['x','y','z'])
    return slave

In [142]:
slave_zeta = slaveEvolution(master,zetaInvar)
slave_null = slaveEvolution(master,zetaNull)

In [143]:
master_trace = go.Scatter3d(name='master',x=master['x'],y=master['y'],z=master['z'],mode='lines')
slave_zeta_trace = go.Scatter3d(name='slave_zeta',x=slave_zeta['x'],y=slave_zeta['y'],z=slave_zeta['z'],mode='lines')
slave_null_trace = go.Scatter3d(name='slave_null',x=slave_null['x'],y=slave_null['y'],z=slave_null['z'],mode='lines')

py.iplot([master_trace,slave_zeta_trace,slave_null_trace])

## synchrony error

In [144]:
def syncError(master,slave,time):
    error = master-slave
    D = (a0+a1*sin(w*time))
    error = error - expand_dims(D,axis=1)
    error = error ** 2
    return error.sum(axis=1)

In [145]:
error_zeta = syncError(master[['x','y','z']].to_numpy(),slave_zeta[['x','y','z']].to_numpy(),master['t'])
error_null = syncError(master[['x','y','z']].to_numpy(),slave_null[['x','y','z']].to_numpy(),master['t'])

In [146]:
error_zeta_trace = go.Scatter(name='zeta',x=master['t'],y=error_zeta)
error_null_trace = go.Scatter(name='null',x=master['t'],y=error_null)
py.iplot([error_zeta_trace,error_null_trace])